In [10]:
from selenium import webdriver
import time 
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
import pandas as pd

In [11]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [12]:
def get_url(key):
    url = f'https://www.amazon.com/s?k={key}&crid=Y3FH4UKLAR0F&sprefix=kitchen+appl%2Caps%2C305&ref=nb_sb_ss_ts-doa-p_2_12'
    return url
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True


In [13]:
driver.get(get_url("kitchen wares"))
link_list = []

n=0
while n < 2:
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a',{'class':'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal'})

    for link in links:
        if '/dp/' in link['href'] and 'https' not in link['href']:
            link_list.append(link['href'])
        else:
            pass
    try:
        element = driver.find_element(By.XPATH, "//a[contains(text(), 'Next')]")
        driver.execute_script('arguments[0].scrollIntoView();', element)
        driver.execute_script('window.scrollBy(0, -200);')
        element.click()
        time.sleep(5)
    except:
        print('No More Pages')
        
    n+=1
    
    
        



In [14]:
prod_info = []
for lin in link_list[:3]:
    prod_dic = {}
    prod_dic['ASIN'] = lin.split('/')[3]
    lin = 'https://www.amazon.com'+lin
    prod_dic['link'] = lin
    driver.get(lin)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        prod_dic['title'] = soup.find('span',{'id':'productTitle'}).text.strip()
        prod_dic['about'] = soup.find('ul',{'class':'a-unordered-list a-vertical a-spacing-mini'}).text
    except:
        continue
    prod_info.append(prod_dic)
    print("Done")
    #break

Done
Done
Done


In [15]:
df = pd.DataFrame(prod_info)

In [16]:
df

,ASIN,link,title,about
0,B0C6QKL8SY,https://www.amazon.com/SODAY-Cookware-Inductio...,"SODAY Pots and Pans Set Non Stick, 12 Pcs Kitc...",Superior Granite Quality: Feature premium gr...
1,B0C5M9W2G5,https://www.amazon.com/Skroam-Containers-Airti...,Skroam 36 Pack Food Storage Containers with li...,All Possible Sizes - This 36pcs plastic airt...
2,B07ZVC6DMM,https://www.amazon.com/40-Piece-Silverware-Sta...,HIWARE 48-Piece Silverware Set with Steak Kniv...,48 PCS SILVERWARE SET FOR 8: Includes 8 dinn...


In [17]:
df.to_csv('testproducts_info2.csv',index=False)